# 处理缺失值

In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pickle
import time

In [2]:
# 修改为您的实际路径
RAW_DATA_PATH = "/root/autodl-tmp/unzipped_data/"  # 原始CSV文件夹路径
PROCESSED_PATH = "./data_clean/"  # 处理结果保存路径
SCALERS_PATH = "./scalers/"  # 标准化器保存路径

In [3]:
import os
import pandas as pd
from glob import glob

# RAW_DATA_PATH = "/root/autodl-tmp/unzipped_data/"
# PROCESSED_PATH = "data/processed/"

def merge_csv_files():
    if os.path.exists(f"{PROCESSED_PATH}merged_data.csv"):
        print("✅ 已存在合并文件，跳过合并步骤")
        return pd.read_csv(f"{PROCESSED_PATH}merged_data.csv")
    
    print("🔄 正在合并CSV文件...")
    file_paths = sorted(glob(f"{RAW_DATA_PATH}*.csv"))
    
    if not file_paths:
        raise FileNotFoundError(f"路径 {RAW_DATA_PATH} 下无CSV文件！")
    
    dfs = []
    for file in file_paths:
        df = pd.read_csv(file)
        date_str = os.path.basename(file).replace(".csv", "")
        date_str_formatted = pd.to_datetime(date_str, format='%Y%m%d').strftime('%Y-%m-%d')
        
        # 提取时间部分并拼接
        df['datetime'] = date_str_formatted + " " + df['datetime'].str.split().str[1]
        
        # 明确指定格式并处理无效值
        df['datetime'] = pd.to_datetime(
            df['datetime'], 
            format='%Y-%m-%d %H:%M:%S',
            errors='coerce'
        )
        
        dfs.append(df)
    
    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df.to_csv(f"{PROCESSED_PATH}merged_data.csv", index=False)
    print(f"✅ 合并完成，已保存至 {PROCESSED_PATH}merged_data.csv")
    return merged_df

#df = merge_csv_files()

In [4]:
print(PROCESSED_PATH)

./data_clean/


In [5]:
df = merge_csv_files()

✅ 已存在合并文件，跳过合并步骤


In [6]:
%%time
def handle_missing_values(df):
    if os.path.exists(f"{PROCESSED_PATH}cleaned_missing.csv"):
        print("✅ 已存在缺失值处理结果，跳过该步骤")
        return pd.read_csv(f"{PROCESSED_PATH}cleaned_missing.csv")
    
    print("🔄 处理缺失值...")
    df.fillna(method='ffill', inplace=True)
    df.dropna(inplace=True)
    df.to_csv(f"{PROCESSED_PATH}cleaned_missing.csv", index=False)
    print(f"✅ 处理完成，已保存至 {PROCESSED_PATH}cleaned_missing.csv")
    return df

df = handle_missing_values(df)

✅ 已存在缺失值处理结果，跳过该步骤
CPU times: user 2.93 s, sys: 611 ms, total: 3.55 s
Wall time: 3.54 s


In [7]:
%%time
def remove_duplicates(df):
    if os.path.exists(f"{PROCESSED_PATH}cleaned_duplicates.csv"):
        print("✅ 已存在去重结果，跳过该步骤")
        return pd.read_csv(f"{PROCESSED_PATH}cleaned_duplicates.csv")
    
    print("🔄 去除重复值...")
    df.drop_duplicates(subset=['datetime'], keep='last', inplace=True)
    df.to_csv(f"{PROCESSED_PATH}cleaned_duplicates.csv", index=False)
    print(f"✅ 去重完成，已保存至 {PROCESSED_PATH}cleaned_duplicates.csv")
    return df

df = remove_duplicates(df)

✅ 已存在去重结果，跳过该步骤
CPU times: user 791 ms, sys: 153 ms, total: 943 ms
Wall time: 941 ms


In [8]:
%%time
def normalize_prices(df):
    if os.path.exists(f"{PROCESSED_PATH}normalized_price.csv"):
        print("✅ 已存在价格归一化结果，跳过该步骤")
        return pd.read_csv(f"{PROCESSED_PATH}normalized_price.csv")
    
    print("🔄 价格归一化处理...")
    scaler = MinMaxScaler()
    price_cols = ['open', 'high', 'low', 'close']
    df[price_cols] = scaler.fit_transform(df[price_cols])
    
    # 保存标准化器
    os.makedirs(SCALERS_PATH, exist_ok=True)
    with open(f"{SCALERS_PATH}scaler_price.pkl", 'wb') as f:
        pickle.dump(scaler, f)
    
    df.to_csv(f"{PROCESSED_PATH}normalized_price.csv", index=False)
    print(f"✅ 价格归一化完成，已保存至 {PROCESSED_PATH}normalized_price.csv")
    return df

df = normalize_prices(df)

✅ 已存在价格归一化结果，跳过该步骤
CPU times: user 964 ms, sys: 95.1 ms, total: 1.06 s
Wall time: 1.06 s


In [9]:
%%time
def normalize_volumes(df):
    if os.path.exists(f"{PROCESSED_PATH}normalized_vol.csv"):
        print("✅ 已存在成交量标准化结果，跳过该步骤")
        return pd.read_csv(f"{PROCESSED_PATH}normalized_vol.csv")
    
    print("🔄 成交量标准化处理...")
    scaler = StandardScaler()
    vol_cols = ['volume', 'openinterest']
    df[vol_cols] = scaler.fit_transform(df[vol_cols])
    
    # 保存标准化器
    with open(f"{SCALERS_PATH}scaler_vol.pkl", 'wb') as f:
        pickle.dump(scaler, f)
    
    df.to_csv(f"{PROCESSED_PATH}normalized_vol.csv", index=False)
    print(f"✅ 成交量标准化完成，已保存至 {PROCESSED_PATH}normalized_vol.csv")
    return df

df = normalize_volumes(df)

✅ 已存在成交量标准化结果，跳过该步骤
CPU times: user 1.15 s, sys: 79.9 ms, total: 1.23 s
Wall time: 1.23 s


In [10]:
df.head()

,datetime,contract,symbol,exchange,open,high,low,close,openinterest,volume,amount
0,2017-01-03 09:00:00,SM1705,SM,CZCE,0.254069,0.249790,0.253348,0.252874,-0.471365,-0.255511,0
1,2017-01-03 09:01:00,SM1705,SM,CZCE,0.253045,0.249035,0.254378,0.253301,-0.471307,-0.276446,0
2,2017-01-03 09:02:00,SM1705,SM,CZCE,0.253642,0.249790,0.255151,0.254069,-0.471259,-0.275214,0
3,2017-01-03 09:03:00,SM1705,SM,CZCE,0.254410,0.251720,0.255580,0.255946,-0.471302,-0.276035,0
4,2017-01-03 09:04:00,SM1705,SM,CZCE,0.255946,0.252223,0.257040,0.256117,-0.471355,-0.262900,0
